In [1]:
import gym_examples
import gym
# from stable_baselines3 import PPO


In [2]:
env = gym.make("gym_examples/StarWars-v0")

In [10]:
start_state = env.reset()
start_state



({'r2d2': array([0, 0]), 'c3po': array([2, 4]), 'vader': array([3, 1])},
 {'c3po_distance': 6.0, 'vader_distance': 4.0})

In [11]:
env.action_space

Dynamic(max_len = 4, current_len = 2, availabe_actions = [0, 3])

In [4]:
env.render()

In [22]:
next_state = env.step(1)
next_state

c:\Users\Eshwar\Documents\GDriveSync\CourseNotes\CSCI566 - Deep Learning\Deep_RL_For_CNN\.env\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


({'r2d2': array([4, 4]), 'c3po': array([0, 1]), 'vader': array([2, 4])},
 -0.005,
 False,
 False,
 {'c3po_distance': 7.0, 'vader_distance': 2.0})